# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [17]:
import numpy as np
import os
#from scipy.misc import imread, imresize
from imageio import imread
from skimage.transform import resize
import datetime
import os

import warnings
warnings.filterwarnings('ignore')

We set the random seed so that the results don't vary drastically.

In [18]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [19]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 40 #experiment with the batch size

In [20]:
# Let we do the generators and input the images as we see that our images have two different sizes. 
x = 30 # No. of frames images
y = 120 # Width of the image
z = 120 # height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # Let us resize all the images 
                    Temp_img = resize(image,(y,z))
                    Temp_img = Temp_img/127.5-1 
                    
                    batch_data[folder,idx,:,:,0] = (Temp_img[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (Temp_img[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (Temp_img[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches.
        
# Let us see that if the folder is not equal to the batch size * num of batches
        
        if (len(folder_list) != batch_size*num_batches):
            print("Batches: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    Temp_img = resize(image,(y,z))
                    Temp_img = Temp_img/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (Temp_img[:,:,0])
                    batch_data[folder,idx,:,:,1] = (Temp_img[:,:,1])
                    batch_data[folder,idx,:,:,2] = (Temp_img[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [21]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30#num_epochs # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


In [22]:
# Let us import all the needed libraries of Keras.
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [23]:
# Let us see the Validate the Losses and put back the checkpoint
model_name = 'model_init_Conv_3D' + '' + str(curr_dt_time).replace(' ','').replace(':','') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_epoch-{epoch:05d}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto', min_lr=0.00001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [24]:
# Let us see that the steps_per_epoch and validation steps are used by fit_generator to decide the no. of next()

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [25]:
# Let us train and validate the model 

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

## Exp 1 With GRU 

In [10]:
# Let us experiment different x,y,z value in the CNNLSTM network and find tune all the image size & Hyperparameters later
x = 16 # number of frames
y = 120 # image width
z = 120 # image height

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_ex24 = Sequential()   
model_ex24.add(TimeDistributed(Conv2D(8, (3, 3),padding='same', activation='relu'),input_shape=(x,y,z,3)))
model_ex24.add(TimeDistributed(BatchNormalization()))
model_ex24.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex24.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model_ex24.add(TimeDistributed(BatchNormalization()))
model_ex24.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex24.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_ex24.add(TimeDistributed(BatchNormalization()))
model_ex24.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex24.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_ex24.add(TimeDistributed(BatchNormalization()))
model_ex24.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex24.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_ex24.add(TimeDistributed(BatchNormalization()))
model_ex24.add(TimeDistributed(MaxPooling2D((2, 2))))

# Flatten layer 

model_ex24.add(TimeDistributed(Flatten()))

model_ex24.add(GRU(64))
model_ex24.add(Dropout(0.25))

# Dense layer 
model_ex24.add(Dense(64,activation='relu'))
model_ex24.add(Dropout(0.25))
# Softmax layer

model_ex24.add(Dense(5, activation='softmax'))

# Adam optimiser

optimiser = Adam(lr=0.001) #write your optimizer
model_ex24.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_ex24.summary())
        


2024-08-02 08:05:14.431063: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-08-02 08:05:14.431148: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5316 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1c:00.0, compute capability: 7.5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 16, 120, 120, 8)  224       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 16, 120, 120, 8)  32        
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 16, 60, 60, 8)    0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 16, 60, 60, 16)   1168      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 16, 60, 60, 16)   6

In [11]:
# Let us train and validate the model 
batch_size = 32
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_ex24.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/30


2024-08-02 08:05:27.386896: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


17/17 [==============================] - ETA: 0s - loss: 1.5038 - categorical_accuracy: 0.3658Source path =  /home/datasets/Project_data/val ; batch size = 32
Batches:  4 Index: 32
17/17 [==============================] - 69s 4s/step - loss: 1.5038 - categorical_accuracy: 0.3658 - val_loss: 1.4960 - val_categorical_accuracy: 0.3958 - lr: 0.0010
Epoch 2/30
12/17 [====================>.........] - ETA: 12s - loss: 1.2044 - categorical_accuracy: 0.5281
Epoch 00002: saving model to model_init_Conv_3D2024-08-02080502.043474/model_epoch-00002.h5
17/17 [==============================] - 42s 3s/step - loss: 1.1986 - categorical_accuracy: 0.5239 - val_loss: 2.4267 - val_categorical_accuracy: 0.0833 - lr: 0.0010
Epoch 3/30
17/17 [==============================] - 41s 3s/step - loss: 1.0752 - categorical_accuracy: 0.5692 - val_loss: 2.8900 - val_categorical_accuracy: 0.1667 - lr: 0.0010
Epoch 4/30
 8/17 [=============>................] - ETA: 15s - loss: 0.9102 - categorical_accuracy: 0.6579
Epoc

## Exp - 2 With GRU model and reduce the dense layer

In [14]:
# Let us experiment different x,y,z value in the CNNLSTM network and find tune all the image size & Hyperparameters later

x = 16 # number of frames
y = 120 # image width
z = 120 # image height

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_ex25 = Sequential()   

model_ex25.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=(x,y,z,3)))
model_ex25.add(TimeDistributed(BatchNormalization()))
model_ex25.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex25.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_ex25.add(TimeDistributed(BatchNormalization()))
model_ex25.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex25.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_ex25.add(TimeDistributed(BatchNormalization()))
model_ex25.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex25.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_ex25.add(TimeDistributed(BatchNormalization()))
model_ex25.add(TimeDistributed(MaxPooling2D((2, 2))))

# Flatten layer 

model_ex25.add(TimeDistributed(Flatten()))

model_ex25.add(GRU(64))
model_ex25.add(Dropout(0.25))

# Dense layer 
model_ex25.add(Dense(64,activation='relu'))
model_ex25.add(Dropout(0.25))
# Softmax layer

model_ex25.add(Dense(5, activation='softmax'))

# Adam optimiser

optimiser = Adam(lr=0.001) #write your optimizer
model_ex25.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_ex25.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_29 (TimeDi  (None, 16, 120, 120, 16)  448      
 stributed)                                                      
                                                                 
 time_distributed_30 (TimeDi  (None, 16, 120, 120, 16)  64       
 stributed)                                                      
                                                                 
 time_distributed_31 (TimeDi  (None, 16, 60, 60, 16)   0         
 stributed)                                                      
                                                                 
 time_distributed_32 (TimeDi  (None, 16, 60, 60, 32)   4640      
 stributed)                                                      
                                                                 
 time_distributed_33 (TimeDi  (None, 16, 60, 60, 32)  

In [15]:
# Let us train and validate the model 
batch_size = 32
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [16]:
model_ex25.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=35, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/35
17/17 [==============================] - ETA: 0s - loss: 1.5704 - categorical_accuracy: 0.3327Source path =  /home/datasets/Project_data/val ; batch size = 32
Batches:  4 Index: 32
17/17 [==============================] - 69s 4s/step - loss: 1.5704 - categorical_accuracy: 0.3327 - val_loss: 1.9652 - val_categorical_accuracy: 0.3333 - lr: 0.0010
Epoch 2/35
12/17 [====================>.........] - ETA: 12s - loss: 1.3044 - categorical_accuracy: 0.4950
Epoch 00002: saving model to model_init_Conv_3D2024-08-02080502.043474/model_epoch-00002.h5
17/17 [==============================] - 42s 3s/step - loss: 1.2969 - categorical_accuracy: 0.4904 - val_loss: 4.0976 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 3/35
17/17 [==============================] - 40s 2s/step - loss: 1.2158 - categorical_accuracy: 0.4961 - val_loss: 3.1074 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 4/35
 8/17 [=============>.

## Exp - 3 with 18 frames and density [ 8,16,32,64,128]

In [29]:
# Let us experiment different x,y,z value in the CNNLSTM network and find tune all the image size & Hyperparameters later

x = 18 # number of frames
y = 120 # image width
z = 120 # image height

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_ex26 = Sequential()  
model_ex26.add(TimeDistributed(Conv2D(8, (3, 3),padding='same', activation='relu'),input_shape=(x,y,z,3)))
model_ex26.add(TimeDistributed(BatchNormalization()))
model_ex26.add(TimeDistributed(MaxPooling2D((2, 2))))

model_ex26.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model_ex26.add(TimeDistributed(BatchNormalization()))
model_ex26.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex26.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_ex26.add(TimeDistributed(BatchNormalization()))
model_ex26.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex26.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_ex26.add(TimeDistributed(BatchNormalization()))
model_ex26.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex26.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_ex26.add(TimeDistributed(BatchNormalization()))
model_ex26.add(TimeDistributed(MaxPooling2D((2, 2))))

# Flatten layer 

model_ex26.add(TimeDistributed(Flatten()))

model_ex26.add(GRU(64))
model_ex26.add(Dropout(0.25))

# Dense layer 
model_ex26.add(Dense(64,activation='relu'))
model_ex26.add(Dropout(0.25))
# Softmax layer

model_ex26.add(Dense(5, activation='softmax'))

# Adam optimiser

optimiser = Adam(lr=0.001) #write your optimizer
model_ex26.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_ex26.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_58 (TimeDi  (None, 18, 120, 120, 8)  224       
 stributed)                                                      
                                                                 
 time_distributed_59 (TimeDi  (None, 18, 120, 120, 8)  32        
 stributed)                                                      
                                                                 
 time_distributed_60 (TimeDi  (None, 18, 60, 60, 8)    0         
 stributed)                                                      
                                                                 
 time_distributed_61 (TimeDi  (None, 18, 60, 60, 16)   1168      
 stributed)                                                      
                                                                 
 time_distributed_62 (TimeDi  (None, 18, 60, 60, 16)  

In [30]:
# Let us train and validate the model 
batch_size = 32
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [31]:
model_ex26.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=40, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/40
17/17 [==============================] - ETA: 0s - loss: 1.5883 - categorical_accuracy: 0.3199Source path =  /home/datasets/Project_data/val ; batch size = 32
Batches:  4 Index: 32
17/17 [==============================] - 76s 5s/step - loss: 1.5883 - categorical_accuracy: 0.3199 - val_loss: 1.5820 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 2/40
 2/17 [==>...........................] - ETA: 1:01 - loss: 1.5315 - categorical_accuracy: 0.2969Batches:  21 Index: 32

Epoch 00002: saving model to model_init_Conv_3D2024-08-02100043.160098/model_epoch-00002.h5
17/17 [==============================] - 48s 3s/step - loss: 1.2306 - categorical_accuracy: 0.5167 - val_loss: 1.7807 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 3/40
15/17 [=========================>....] - ETA: 5s - loss: 1.1459 - categorical_accuracy: 0.5565
Epoch 00003: saving model to model_init_Conv_3D2024-08-02100043.160098/model_epo

## Exp - 4 with 16 frames and density [8,16,32,64,128]

In [32]:
# Let us experiment different x,y,z value in the CNNLSTM network and find tune all the image size & Hyperparameters later

x = 16 # number of frames
y = 120 # image width
z = 120 # image height

# Input all the images sequencial by building the layer with dropouts and batchnormalisation

model_ex30 = Sequential()   
model_ex30.add(TimeDistributed(Conv2D(8, (3, 3),padding='same', activation='relu'),input_shape=(x,y,z,3)))
model_ex30.add(TimeDistributed(BatchNormalization()))
model_ex30.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex30.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu')))
model_ex30.add(TimeDistributed(BatchNormalization()))
model_ex30.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex30.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_ex30.add(TimeDistributed(BatchNormalization()))
model_ex30.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex30.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_ex30.add(TimeDistributed(BatchNormalization()))
model_ex30.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_ex30.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_ex30.add(TimeDistributed(BatchNormalization()))
model_ex30.add(TimeDistributed(MaxPooling2D((2, 2))))

# Flatten layer 

model_ex30.add(TimeDistributed(Flatten()))

model_ex30.add(GRU(128))
model_ex30.add(Dropout(0.25))

# Dense layer 
model_ex30.add(Dense(128,activation='relu'))
model_ex30.add(Dropout(0.25))
# Softmax layer

model_ex30.add(Dense(5, activation='softmax'))

# Adam optimiser

optimiser = Adam(lr=0.001) #write your optimizer
model_ex30.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_ex30.summary())
        

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_74 (TimeDi  (None, 16, 120, 120, 8)  224       
 stributed)                                                      
                                                                 
 time_distributed_75 (TimeDi  (None, 16, 120, 120, 8)  32        
 stributed)                                                      
                                                                 
 time_distributed_76 (TimeDi  (None, 16, 60, 60, 8)    0         
 stributed)                                                      
                                                                 
 time_distributed_77 (TimeDi  (None, 16, 60, 60, 16)   1168      
 stributed)                                                      
                                                                 
 time_distributed_78 (TimeDi  (None, 16, 60, 60, 16)  

In [34]:
# Let us train and validate the model 
batch_size = 40
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [35]:
model_ex30.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 40
Epoch 1/30
17/17 [==============================] - ETA: 0s - loss: 1.5139 - categorical_accuracy: 0.3379Source path =  /home/datasets/Project_data/val ; batch size = 40
Batches:  3 Index: 40
17/17 [==============================] - 90s 5s/step - loss: 1.5139 - categorical_accuracy: 0.3379 - val_loss: 1.4354 - val_categorical_accuracy: 0.4200 - lr: 0.0010
Epoch 2/30
12/17 [====================>.........] - ETA: 12s - loss: 1.1033 - categorical_accuracy: 0.5580
Epoch 00002: saving model to model_init_Conv_3D2024-08-02100043.160098/model_epoch-00002.h5
17/17 [==============================] - 50s 3s/step - loss: 1.0886 - categorical_accuracy: 0.5729 - val_loss: 1.5317 - val_categorical_accuracy: 0.3500 - lr: 0.0010
Epoch 3/30
17/17 [==============================] - 47s 3s/step - loss: 1.1111 - categorical_accuracy: 0.5504 - val_loss: 1.8919 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 4/30
 8/17 [=============>.